# EEG data from basic sensory task in Schizophrenia

Authors: Karl Jääts, Mihkel Kruusi, Siim K. Koger

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib notebook

In [7]:
demo = pd.read_csv("demographic.csv")
erp_data = pd.read_csv("ERPdata.csv")
display(erp_data[:4])

,subject,condition,Fz,FCz,Cz,FC3,FC4,C3,C4,CP3,CP4,time_ms
0,1,1,5.533701,5.726507,5.469535,5.386723,4.588875,6.560092,4.542811,5.397492,5.103695,-1500.0000
1,1,1,5.651489,5.837326,5.773131,5.627975,4.822217,6.739976,4.811770,5.541357,5.379273,-1499.0234
2,1,1,5.717580,5.932924,5.948466,5.826460,4.979647,7.026199,5.053779,5.634972,5.600504,-1498.0469
3,1,1,5.703267,5.968103,5.851512,5.812192,4.992899,6.940671,5.106650,5.543577,5.589775,-1497.0703


In [45]:
BL = erp_data[(erp_data.time_ms >= -100) & (erp_data.time_ms <= 0)].groupby(['subject', 'condition']).mean()

N1amps = erp_data[(erp_data.time_ms >= 80) & (erp_data.time_ms <= 100)].groupby(['subject', 'condition']).mean()

P2amps = erp_data[(erp_data.time_ms >= 150) & (erp_data.time_ms <= 190)].groupby(['subject', 'condition']).mean()

display(BL[:4])
display(N1amps[:4])
N1amps[["Fz", "FCz", "Cz"]] = N1amps[["Fz", "FCz", "Cz"]] - BL[["Fz", "FCz", "Cz"]]
display(N1amps[:4])

P2amps[["Fz", "FCz", "Cz"]] = P2amps[["Fz", "FCz", "Cz"]] - BL[["Fz", "FCz", "Cz"]]

Fz       FCz        Cz       FC3       FC4        C3  \
subject condition                                                               
1       1          0.008189  0.009720  0.016123  0.010271  0.008870  0.020370   
        2         -0.006360 -0.005304 -0.003952 -0.003845 -0.006621 -0.001391   
        3         -0.010922 -0.010625 -0.005603 -0.013810 -0.009102 -0.004477   
2       1          0.003162  0.001844  0.004312  0.002926  0.001148  0.005238   

                         C4       CP3       CP4    time_ms  
subject condition                                           
1       1          0.011190  0.018696  0.012184 -49.804694  
        2         -0.002625 -0.005219 -0.001973 -49.804694  
        3         -0.003762  0.001899  0.000354 -49.804694  
2       1          0.004516  0.005816  0.004715 -49.804694

Fz       FCz        Cz       FC3       FC4        C3  \
subject condition                                                               
1       1         -3.969841 -3.531790 -2.742199 -2.411903 -4.154254 -1.621322   
        2         -8.308424 -8.990923 -9.474523 -8.171511 -8.710126 -8.271828   
        3          0.967277  1.329361  1.825440  1.013117  1.553381  1.351172   
2       1         -2.852217 -2.915010 -2.492622 -1.702627 -2.906801 -1.227084   

                         C4       CP3       CP4    time_ms  
subject condition                                           
1       1         -3.716837 -2.547987 -3.497883  89.843757  
        2         -8.220033 -8.231913 -8.064169  89.843757  
        3          1.075092  1.737629  0.796994  89.843757  
2       1         -2.110256 -0.918345 -1.365077  89.843757

Fz       FCz        Cz       FC3       FC4        C3  \
subject condition                                                               
1       1         -3.978030 -3.541510 -2.758322 -2.411903 -4.154254 -1.621322   
        2         -8.302064 -8.985619 -9.470571 -8.171511 -8.710126 -8.271828   
        3          0.978199  1.339986  1.831042  1.013117  1.553381  1.351172   
2       1         -2.855379 -2.916854 -2.496934 -1.702627 -2.906801 -1.227084   

                         C4       CP3       CP4    time_ms  
subject condition                                           
1       1         -3.716837 -2.547987 -3.497883  89.843757  
        2         -8.220033 -8.231913 -8.064169  89.843757  
        3          1.075092  1.737629  0.796994  89.843757  
2       1         -2.110256 -0.918345 -1.365077  89.843757